In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 3, Finished, Available)

In [2]:
DeltaTable.createIfNotExists(spark)\
        .tableName("Gold_DimOrderPriority")\
        .addColumn("OrderPriority_ID",LongType())\
        .addColumn("Order_Priority",StringType())\
        .addColumn("Created_TS",TimestampType())\
        .addColumn("Modified_TS",TimestampType())\
        .execute()

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 4, Finished, Available)

In [3]:
df = spark.read.table('lh_ecormece.Gold_DimOrderPriority')

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 5, Finished, Available)

In [9]:
max_date=df.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 11, Finished, Available)

In [10]:
max_date

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 12, Finished, Available)

'1900-01-01'

In [11]:
df_bronze=spark.read.table("lh_ecormece.bronze_sales")

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 13, Finished, Available)

In [30]:
df_bronze_mod=df_bronze.select("Order_Priority").where(col("Modified_TS")>max_date).drop_duplicates()

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 32, Finished, Available)

In [31]:
max_ID=df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]

df_final=df_bronze_mod.withColumn("OrderPriority_ID",max_ID+monotonically_increasing_id()+1)

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 33, Finished, Available)

In [32]:
df_gold_delta=DeltaTable.forPath(spark,"Tables/gold_dimorderpriority")
df_bonze_table=df_final

StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 34, Finished, Available)

In [33]:
df_gold_delta.alias("gold")\
             .merge(\
               df_bonze_table.alias("bronze"),\
                "gold.Order_Priority==Bronze.Order_Priority"\
             )\
             .whenMatchedUpdate(\
                  set={
                    "gold.Modified_TS":current_timestamp()
                  }
             )\
             .whenNotMatchedInsert(\
                    values={
                     "gold.OrderPriority_ID":"Bronze.OrderPriority_ID",
                     "gold.Order_Priority":"Bronze.Order_Priority",
                     "gold.Created_TS":current_timestamp(),
                     "gold.Modified_TS":current_timestamp()
                    }
             )\
             .execute()


StatementMeta(, 783fc8f7-bf8c-4475-a9b5-16aacdd6bf81, 35, Finished, Available)